In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
url = "https://www.zalando.es/catalogo/nike-performance.puma.reebok.tyr.under-armour/?q=zapatillas+crossfit&sport_type=entrenamiento-y-fitness"


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com'
}
response = requests.get(url,headers=headers)
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.content, "html.parser")

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

def extract_product_data(url):
    driver = webdriver.Chrome()

    url = "https://www.zalando.es/catalogo/nike-performance.puma.reebok.tyr.under-armour/?q=zapatillas+crossfit&sport_type=entrenamiento-y-fitness"

    # Abrir la página con Selenium
    driver.get(url)
    
    # Esperar un tiempo para que cargue el contenido dinámico (puedes ajustar el tiempo)
    time.sleep(3)

    # Obtener el HTML de la página
    html = driver.page_source
    
    # Crear el objeto BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Crear DataFrame vacío con las columnas deseadas
    df = pd.DataFrame(columns=['date', 'time', 'model', 'color', 'current_price', 'normal_price', 'link'])

    # Buscar los productos en la página usando BeautifulSoup
    product_titles = soup.find_all('h3', class_='sDq_FX lystZ1 FxZV-M HlZ_Tf ZkIJC- r9BRio qXofat EKabf7')
    current_prices = soup.find_all('span', class_='sDq_FX lystZ1 dgII7d Km7l2y')
    normal_prices = soup.find_all('span', class_='sDq_FX goVnUa FxZV-M Yb63TQ ZiDB59 uVxVjw')
    product_links = soup.find_all('a', class_='q84f1m CKDt_l LyRfpJ JT3_zV CKDt_l _2dqvZS')

    # Determinar el número de productos (basado en títulos)
    num_items = len(product_titles)

    # Obtener la fecha y hora actuales
    current_datetime = datetime.now()
    current_date = current_datetime.strftime('%Y-%m-%d')
    current_time = current_datetime.strftime('%H:%M:%S')

    # Iterar sobre las tarjetas de productos y extraer información
    for i in range(num_items):
        # Pausa para evitar ser bloqueado por el servidor
        time.sleep(1)

        # Obtener el título completo del producto
        full_title = product_titles[i].text.strip() if i < len(product_titles) else "N/A"

        # Extraer modelo (primera parte antes del primer "-")
        model = full_title.split('-')[0].strip() if '-' in full_title else full_title

        # Extraer color (parte después del último "-")
        color = full_title.split('-')[-1].strip() if '-' in full_title else "N/A"

        # Obtener el precio actual
        current_price = current_prices[i].text.strip() if i < len(current_prices) else "N/A"

        # Obtener el precio normal (verificar si existe, si no, asignar "N/A")
        normal_price = normal_prices[i].text.strip() if i < len(normal_prices) else "N/A"

        # Si no se encontró en la clase original, buscar en la clase alternativa
        if normal_price == "N/A":
            alt_price_tag = product_titles[i].find_next('span', class_='sDq_FX lystZ1 FxZV-M HlZ_Tf')
            normal_price = alt_price_tag.text.strip() if alt_price_tag else "N/A"

        # Obtener el link del producto
        link = product_links[i]['href'] if i < len(product_links) else "N/A"

        # Agregar los datos al DataFrame
        df.loc[i] = [current_date, current_time, model, color, current_price, normal_price, link]

    # Cerrar el navegador
    driver.quit()

    # Eliminar filas con valores nulos en current_price y normal_price
    df = df.dropna(subset=['current_price', 'normal_price'])
    
    # Resetear el índice después de eliminar filas
    df = df.reset_index(drop=True)

    return df

In [3]:
df=extract_product_data(soup)

In [4]:
# Obtener la fecha y hora actual
fecha_hora = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Crear el nombre del archivo con la fecha y hora
nombre_archivo = f"productos_zalando_{fecha_hora}.csv"

# Guardar el archivo CSV
df.to_csv(rf'data\{nombre_archivo}', index=False, encoding='utf-8-sig')